In [1]:
import tensorflow as tf
import settings
import models
import numpy as np
import scipy.misc


In [7]:
def loss(sess,model):
    #define ith layer as content loss
    content_layers = settings.CONTENT_LOSS_LAYERS
    #assign content as input
    sess.run(tf.assign(model.model['input'], model.content))
    content_loss = 0.0
    for layer_name, weight in content_layers:
        #get feature matrix of content img
        p = sess.run(model.model[layer_name])
        #get feature matrix of noise img
        x = model.model[layer_name]
        #get size and channel
        M,N= p.shape[1]*p.shape[2],p.shape[3]
        
        content_loss += (1.0 / (2 * M * N)) * tf.reduce_sum(tf.pow(p - x, 2)) * weight
    
    content_loss /= len(content_layers) #take avg
    
    
    style_layers = settings.STYLE_LOSS_LAYERS
    sess.run(tf.assign(model.model['input'], model.style))
    style_loss = 0.0
    for layer_name, weight in style_layers:
        a = sess.run(model.model[layer_name])
        #get feature matrix of noise img
        x = model.model[layer_name]
        #get size and channel
        M,N= a.shape[1]*a.shape[2],a.shape[3]
        #Gram matrix of a 
        A = grammatrix(a,M,N)
        #Gram matrix of x 
        G = grammatrix(x,M,N)
        
        style_loss += (1.0 / (4 * M * M * N * N)) * tf.reduce_sum(tf.pow(G - A, 2)) * weight
    style_loss /= len(style_layers)
    
    loss = settings.ALPHA * content_loss + settings.BETA * style_loss

    return loss
        
        
        
def grammatrix(x,size,channel):
    '''grammatrix define how corelatted among activated channels in Lth layer
    kind of like covariance matrix'''
    x = tf.reshape(x,(size,channel)) 
    g = tf.matmul(tf.transpose(x), x) # g of NC*NC
    return g

def train(iteration=500):
    model = models.Model(settings.CONTENT_IMAGE,settings.STYLE_IMAGE)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        cost = loss(sess, model)
        optimizer = tf.train.AdamOptimizer(1.0).minimize(cost)
        sess.run(tf.global_variables_initializer())
        sess.run(tf.assign(model.model['input'], model.random_img))
    
        for i in range(iteration):
            sess.run(optimizer)
            if i%50==0:
                print ('step {} is down. Loss:{}'.format(i,sess.run(cost)))
                # get the imgs updated
                img = sess.run(model.model['input'])
                # add back mean
                img += settings.IMAGE_MEAN_VALUE
                # batch[0]
                img = img[0]
                # clip outrange nums
                img = np.clip(img, 0, 255).astype(np.uint8)
                # save img
                scipy.misc.imsave('{}-{}.jpg'.format(settings.OUTPUT_IMAGE,i), img)
    
        # save results
        img = sess.run(model.model['input'])
        img += settings.IMAGE_MEAN_VALUE
        img = img[0]
        img = np.clip(img, 0, 255).astype(np.uint8)
        scipy.misc.imsave('{}.jpg'.format(settings.OUTPUT_IMAGE), img)
        




if __name__ == '__main__':
    train(iteration=500)


step 0 is down. Loss is 8985040896.0


/Users/pisco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


KeyboardInterrupt: 